## Deliverable 2. Create a Customer Travel Destinations Map.

In [45]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [33]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Honiara,SB,-9.4333,159.9500,84.67,71,61,3.85,broken clouds
1,1,Nicoya,CR,10.1483,-85.4520,71.60,94,75,5.75,broken clouds
2,2,Pevek,RU,69.7008,170.3133,29.16,65,83,15.50,broken clouds
3,3,Oktyabrskiy,RU,54.4815,53.4710,52.09,86,98,6.44,overcast clouds
4,4,Albany,US,42.6001,-73.9662,59.00,77,90,5.75,overcast clouds


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Honiara,SB,-9.4333,159.9500,84.67,71,61,3.85,broken clouds
1,1,Nicoya,CR,10.1483,-85.4520,71.60,94,75,5.75,broken clouds
6,6,Butaritari,KI,3.0707,172.7902,83.73,77,72,19.73,light rain
8,8,Rikitea,PF,-23.1203,-134.9692,78.57,79,99,10.96,light rain
9,9,Teguise,ES,29.0605,-13.5640,66.20,63,20,12.66,few clouds
10,10,Vao,NC,-22.6667,167.4833,73.58,79,49,17.11,scattered clouds
16,16,Arraial Do Cabo,BR,-22.9661,-42.0278,69.80,85,0,4.61,clear sky
19,19,Hilo,US,19.7297,-155.0900,84.20,54,20,11.50,light rain
21,21,Georgetown,MY,5.4112,100.3354,79.00,94,20,5.23,few clouds
22,22,Alta Floresta,BR,-9.8756,-56.0861,84.20,61,20,3.44,few clouds


In [36]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()



City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Honiara,SB,84.67,broken clouds,-9.4333,159.9500,
1,Nicoya,CR,71.60,broken clouds,10.1483,-85.4520,
6,Butaritari,KI,83.73,light rain,3.0707,172.7902,
8,Rikitea,PF,78.57,light rain,-23.1203,-134.9692,
9,Teguise,ES,66.20,few clouds,29.0605,-13.5640,
10,Vao,NC,73.58,scattered clouds,-22.6667,167.4833,
16,Arraial Do Cabo,BR,69.80,clear sky,-22.9661,-42.0278,
19,Hilo,US,84.20,light rain,19.7297,-155.0900,
21,Georgetown,MY,79.00,few clouds,5.4112,100.3354,
22,Alta Floresta,BR,84.20,few clouds,-9.8756,-56.0861,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [53]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [54]:
clean_hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Honiara,SB,84.67,broken clouds,-9.4333,159.9500,Heritage Park Hotel
1,Nicoya,CR,71.60,broken clouds,10.1483,-85.4520,Guayacan Real
6,Butaritari,KI,83.73,light rain,3.0707,172.7902,Isles Sunset Lodge
8,Rikitea,PF,78.57,light rain,-23.1203,-134.9692,Pension Maro'i
9,Teguise,ES,66.20,few clouds,29.0605,-13.5640,B&B La Casona de Nazaret
10,Vao,NC,73.58,scattered clouds,-22.6667,167.4833,Hotel Kou-Bugny
16,Arraial Do Cabo,BR,69.80,clear sky,-22.9661,-42.0278,Pousada Porto Praia
19,Hilo,US,84.20,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
21,Georgetown,MY,79.00,few clouds,5.4112,100.3354,Cititel Penang
22,Alta Floresta,BR,84.20,few clouds,-9.8756,-56.0861,Floresta Amazonica Hotel


In [55]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))